In [66]:
import pandas as pd

In [67]:
from bs4 import BeautifulSoup
import requests

import selenium
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [69]:
#bs4 & requests
webpage = requests.get('https://eyewa.com/ae-en/', 'html.parser')
soup = BeautifulSoup(webpage.content)

#find the header
header_menu = soup.find(attrs={"id": "mainMenu"}).find_all('li', attrs={"class":["mega-menu-item mega-menu-fullwidth menu-3columns level0 static-menu level0 dropdown","mega-menu-item mega-menu-fullwidth menu-2columns level0 static-menu level0 dropdown"]} )

#input the section into a dict
sections_dict = {}
for section in header_menu:
    print(section.a.span.string, section.a['href'])
    sections_dict[section.a.span.string] = section.a['href']

print(sections_dict)

    

Sunglasses https://eyewa.com/ae-en/sunglasses.html
Glasses https://eyewa.com/ae-en/glasses.html
Blue Light Glasses https://eyewa.com/ae-en/blue-light-glasses.html
Corrective Contact Lenses https://eyewa.com/ae-en/corrective-contact-lenses.html
Color Contact Lenses https://eyewa.com/ae-en/color-contact-lenses.html
Clearance https://eyewa.com/ae-en/clearance-sale.html
{'Sunglasses': 'https://eyewa.com/ae-en/sunglasses.html', 'Glasses': 'https://eyewa.com/ae-en/glasses.html', 'Blue Light Glasses': 'https://eyewa.com/ae-en/blue-light-glasses.html', 'Corrective Contact Lenses': 'https://eyewa.com/ae-en/corrective-contact-lenses.html', 'Color Contact Lenses': 'https://eyewa.com/ae-en/color-contact-lenses.html', 'Clearance': 'https://eyewa.com/ae-en/clearance-sale.html'}


In [70]:
#get all category pages html
for x,y in sections_dict.items():
    page = requests.get(y,'html.parser')


In [71]:
def product_getter(all_products_dict, link):
    print(f'getting data from {link}')
    webpage = requests.get(link, 'html.parser')
    soup = BeautifulSoup(webpage.content)
    try:
        product_list =  soup.find(attrs={'class':"products list items product-items row row-col-lg-3"})
        for i in product_list.find_all('li'): 
            product_top =i.find(attrs={'class':'product-top'})
            product_url = product_top.a['href']
            product_top = product_top.a.find('img')
            product_name = product_top['alt']
            product_picture = product_top['src']
            
            all_products_dict[product_name]={}
            all_products_dict[product_name]['product_url']=product_url
            all_products_dict[product_name]['picture_url']=product_picture
    except Exception as e:
        print(e)


def next_page_finder(current_link):
    try:
        next_link = requests.get(current_link, 'html.parser')
        next_link = BeautifulSoup(next_link.content)
        next_link = next_link.find('li',attrs={'class':'item pages-item-next'}).a['href']
        return next_link

    except Exception as e:
        print('End of the ride for this category')
        return True

def product_dataframe(all_products_dict):
    df = pd.DataFrame.from_dict(all_products_dict,orient='index')
    print(df.head())
    return df


In [72]:
webpage = requests.get('https://eyewa.com/ae-en/', 'html.parser')
soup = BeautifulSoup(webpage.content)
#find the header
header_menu = soup.find(attrs={"id": "mainMenu"}).find_all('li', attrs={"class":["mega-menu-item mega-menu-fullwidth menu-3columns level0 static-menu level0 dropdown","mega-menu-item mega-menu-fullwidth menu-2columns level0 static-menu level0 dropdown"]} )

#input the section into a dict
sections_dict = {}
for section in header_menu:
    print(section.a.span.string, section.a['href'])
    sections_dict[section.a.span.string] = section.a['href']

all_products_dict = {}
for x,y in sections_dict.items():
    next_link=y
    while next_link != True:
        product_getter(all_products_dict, next_link)
        next_link = next_page_finder(next_link)

df = product_dataframe(all_products_dict)


Sunglasses https://eyewa.com/ae-en/sunglasses.html
Glasses https://eyewa.com/ae-en/glasses.html
Blue Light Glasses https://eyewa.com/ae-en/blue-light-glasses.html
Corrective Contact Lenses https://eyewa.com/ae-en/corrective-contact-lenses.html
Color Contact Lenses https://eyewa.com/ae-en/color-contact-lenses.html
Clearance https://eyewa.com/ae-en/clearance-sale.html
getting data from https://eyewa.com/ae-en/sunglasses.html
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=2
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=3
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=4
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=5
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=6
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=7
getting data from https://eyewa.com/ae-en/sunglasses.html?html_parser=&p=8
getting data from https://eyewa.com/ae-en/sungla

In [191]:
df1 = df.copy(deep=True)

In [194]:
df.iloc[:4].index

Index(['BlackOut MIDNIGHT', 'BlackOut SERENE', 'BlackOut MOONLIGHT',
       'BlackOut COACHELLA'],
      dtype='object')

In [274]:
product_details = {}
#for ind in df.iloc[:4].index:
for ind in df.index:
    print('********************************************************************************************************')
    product_details[ind]={}
    current_link = df['product_url'][ind]
    product_details[ind]['product_url']= current_link 

    print(f'getting data from {current_link}')
    html = requests.get(current_link)
    if html.status_code == 404:
        continue
    else:
        print(f'html response: {html.status_code}')
    
    links_not_working = {}

    soup = BeautifulSoup(html.content)

    retries=0
    if  soup.find(attrs={'class':'cf-error-type'}) != None :
        if retries < 5:
            print(soup.find(attrs={'class':'cf-error-type'}))
            print('retrying')
            html = requests.get(current_link)
            soup = BeautifulSoup(html.content)
            retries += 1
    
        else:
            print(f"Current link didn't work: {current_link}")
            links_not_working[ind] = current_link
            continue

    
    try:
        id = soup.find(attrs={'class':'price-box price-final_price'})['data-product-id']
        product_details[ind]['product_id'] = id
    except:
        print("couldnt find product id")
        print(soup.find(attrs={'class':'price-box price-final_price'}))

    retries = 0    
    if soup.find(attrs={'id':'productPriceInfo'}) is None:
        if retries < 5:
            print('retrying')
            html = requests.get(current_link)
            soup = BeautifulSoup(html.content)
            retries += 1
        else:
            continue
    else:
        price =  soup.find(attrs={'id':'productPriceInfo'})
        print(f'found tag for price')
        try:
            oldPrice = price.find(attrs={'class':'price-wrapper','data-price-type':'oldPrice'})['data-price-amount']
            currentPrice = price.find(attrs={'class':'price-wrapper','data-price-type':'finalPrice'})['data-price-amount']
            product_details[ind]['discounted'] = 'yes'
            product_details[ind]['discountedPrice'] = currentPrice
            product_details[ind]['actualPrice'] = oldPrice
            print(f'101: \n {price}')

        except Exception as e:
            print(e)
            print(price)
            currentPrice = price.find(attrs={'class':'price-wrapper','data-price-type':'finalPrice'})['data-price-amount']   
            product_details[ind]['discounted'] = 'no'
            product_details[ind]['actualPrice'] = currentPrice



    table = soup.find(attrs={'class':'data table additional-attributes'}).find('tbody')
    rows = table.find_all('td')
    print(f'attributes:\n{rows}')
    for row in range(len(rows)):
        #print(rows[row]['data-th'],rows[row].string)
        product_details[ind][rows[row]['data-th']] = rows[row].string
        print(ind, rows[row].string)

    #print(product_details[ind])

********************************************************************************************************
getting data from https://eyewa.com/ae-en/blackout-midnight-000215-0707.html
html response: 200
found tag for price
'NoneType' object is not subscriptable
<div class="product-info-price" id="productPriceInfo"><div class="price-box price-final_price" data-price-box="product-id-69406" data-product-id="69406" data-role="priceBox">
<span class="price-container price-final_price tax weee">
<span class="price-wrapper" data-price-amount="199" data-price-type="finalPrice" id="product-price-69406">
<span class="price">AED 199</span> </span>
</span>
</div></div>
attributes:
[<td class="col data" data-th="Frame Finish">Glossy</td>, <td class="col data" data-th="Lens Finish">Gradient</td>, <td class="col data" data-th="Lens Color">Gray</td>, <td class="col data" data-th="Brand">BlackOut</td>, <td class="col data" data-th="Frame Color">Gray</td>, <td class="col data" data-th="Frame Material">Met

In [275]:
df2 =pd.DataFrame.from_dict(product_details,orient='index')
df2.head()

,product_url,product_id,discounted,actualPrice,Frame Finish,Lens Finish,Lens Color,Brand,Frame Color,Frame Material,...,Diameter,Base Curve,Water Content,Oxygen Permeability,Lens Material,UV Protection,Contact Lens Replacement,Contact Lens Type,Contact Lens Use,Package Size
BlackOut MIDNIGHT,https://eyewa.com/ae-en/blackout-midnight-0002...,69406,no,199,Glossy,Gradient,Gray,BlackOut,Gray,Metal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BlackOut SERENE,https://eyewa.com/ae-en/blackout-serene-000107...,59934,no,199,Glossy,Gradient,Gray,BlackOut,Black,Plastic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BlackOut MOONLIGHT,https://eyewa.com/ae-en/blackout-moonlight-000...,69404,no,199,Glossy,Gradient,Green,BlackOut,Gold,Metal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BlackOut COACHELLA,https://eyewa.com/ae-en/blackout-coachella-000...,62404,yes,199,Glossy,Gradient,Gray,BlackOut,Gold,Metal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30Sundays SMOOTH SAILING,https://eyewa.com/ae-en/30sundays-smooth-saili...,59125,yes,369,Glossy,Gradient,Gray,30Sundays,Silver,Metal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
df2.describe()

,product_url,product_id,discounted,actualPrice,Frame Finish,Lens Finish,Lens Color,Brand,Frame Color,Frame Material,...,Diameter,Base Curve,Water Content,Oxygen Permeability,Lens Material,UV Protection,Contact Lens Replacement,Contact Lens Type,Contact Lens Use,Package Size
count,718,704,704,704,263,128,451,714,457,465,...,233,234,234,50,54,54,249,249,249,210
unique,718,550,2,93,2,3,11,62,21,14,...,9,8,19,26,19,2,6,2,3,5
top,https://eyewa.com/ae-en/bella-elite-2021-colle...,12745,no,399,Glossy,Full,Clear,Spectus,Black,Acetate,...,14.2 mm,8.6,38%,27 Dk/t,Senofilcon A,Yes,Daily,Color Contact Lenses,Beauty,Pack of 2 lenses
freq,1,17,504,146,210,90,282,134,138,180,...,123,196,113,4,8,29,70,190,190,163


In [277]:
df.describe()

,product_url,picture_url
count,718,718
unique,718,1
top,https://eyewa.com/ae-en/bella-elite-2021-colle...,https://eyewa.com/static/version1630831330/fro...
freq,1,718


In [278]:
df2.to_csv('eywa_datapull.csv')